In [1]:
import requests
import csv
import re
import pandas as pd
import time
from bs4 import BeautifulSoup
import os
from numpy import arange

In [9]:
all_resturants = 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=western%20sydney&l=g%3A151.15106940214844%2C-33.67728783357405%2C150.7418286794922%2C-34.01944832216618&ed_attrs=RestaurantsPriceRange2.1'

In [10]:
inexp_page1 = 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=western%20sydney&l=g%3A151.15106940214844%2C-33.67728783357405%2C150.7418286794922%2C-34.01944832216618&attrs=RestaurantsPriceRange2.1'

In [11]:
mod_page1 = 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=western%20sydney&l=g%3A151.15106940214844%2C-33.67728783357405%2C150.7418286794922%2C-34.01944832216618&attrs=RestaurantsPriceRange2.2&ed_attrs=RestaurantsPriceRange2.1%2CRestaurantsPriceRange2.2'

In [13]:
pricey_page1 = 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=western%20sydney&l=g%3A151.15106940214844%2C-33.67728783357405%2C150.7418286794922%2C-34.01944832216618&ed_attrs=RestaurantsPriceRange2.1%2CRestaurantsPriceRange2.2&attrs=RestaurantsPriceRange2.3'

In [14]:
inexp_page1

'https://www.yelp.com/search?find_desc=Restaurants&find_loc=western%20sydney&l=g%3A151.15106940214844%2C-33.67728783357405%2C150.7418286794922%2C-34.01944832216618&attrs=RestaurantsPriceRange2.1'

In [15]:
inexp_page2='https://www.yelp.com/search?find_desc=Restaurants&find_loc=western%20sydney&l=g%3A151.15106940214844%2C-33.67728783357405%2C150.7418286794922%2C-34.01944832216618&attrs=RestaurantsPriceRange2.1&start=30'

In [16]:
inexp_page2

'https://www.yelp.com/search?find_desc=Restaurants&find_loc=western%20sydney&l=g%3A151.15106940214844%2C-33.67728783357405%2C150.7418286794922%2C-34.01944832216618&attrs=RestaurantsPriceRange2.1&start=30'

In [17]:
inexp_page2_firstrest = 'https://www.yelp.com/biz/mr-pings-parramatta?hrid=fJdzg_w2qK4Omjp4h0qc1g&osq=Restaurants'

In [ ]:
rashays = 'https://www.yelp.com/biz/rashays-lidcombe?osq=Restaurants'

In [ ]:
mrpings = 'https://www.yelp.com/biz/mr-pings-parramatta?osq=Restaurants'

In [ ]:
eljannah = 'https://www.yelp.com/biz/el-jannah-granville-granville?hrid=8RlpHgyMo6ryt_Q1ebNNGQ&osq=Restaurants'

In [18]:
eljannah_page2_reviews = 'https://www.yelp.com/biz/el-jannah-granville-granville?start=20'

***Pages of reviews for each resturant*** <br>
1) We can use all characters between "datePublished.....and... /iv>" to capture the actual texts of reviews <br>
2) We can also use "ratingValue" content=" as a marker for star rating <br>
3) we will have to use the total number of reviews (found in plain html on the first page of all resturants) to calculate how many pages of reviews (assuming 20 reviews per page) 

***Pages of resturants for each price point***<br>
1) We can calculate total pages by using the total number of resturants (found in plain html at the top of each price point page) and then we assume 30 resturants per page <br>
2) We will have to also scrape all resturant names first and convert to all small case with dashes put between each word and punctuation marks removed in order to construct URLs for individual resturants for pre-review scraping <br>
3) We can use "ranking"....."name"......"neighbohoods" as markers for each resturant name we have to scrape and process

***Scrapper Pseudo Code***<br>
1- Calculate the number of pages of resturants each price point has <br>
2- Extract and process all resturant names for each price point <br>
3- Calculate the number of pages of reviews each resturant has <br>
4- Use numbers from above to establish dataframes and loops <br>
5- Finally scrape all reviews and star ratings into 4 seperate dataframes, one for each price point </br>